In [1]:
import data
import models
import cache
import seaborn as sns
import numpy as np
import pandas as pd
import patsy
from matplotlib import pyplot as plt

/home/jacquelineburos/miniconda3/envs/immuneinf/lib/python3.5/site-packages/Cython/Distutils/old_build_ext.py:30: UserWarning: Cython.Distutils.old_build_ext does not properly handle dependencies and is deprecated.
  "Cython.Distutils.old_build_ext does not properly handle dependencies "


INFO:stancache.seed:Setting seed to 1245502385


INFO:root:Setting CACHE_DIR = /mnt/modelcache/immune-infiltrate-explorations


INFO:stancache.seed:Setting seed to 1245502385


In [2]:
sns.set(context='talk')

In [3]:
model_name = 'model5'
by = 'SubSet'
sample_n = 1000

## load files for all cell types

In [4]:
df = cache.cached(data.prep_annotated_data)

INFO:stancache.stancache:prep_annotated_data: cache_filename set to prep_annotated_data.cached.default.pkl


INFO:stancache.stancache:prep_annotated_data: Loading result from cache


In [5]:
assert all(pd.notnull(df['log1p_tpm_rescaled']))

In [6]:
print(df.columns)

#apply(lambda x: x.startswith('C'))

Index(['sample_id', 'filename', 'gene_name', 'est_counts', 'tpm', 'log1p_tpm',
       'log1p_counts', 'CCR6+', 'CCR7+', 'CCR7-', 'CD127-', 'CD161+', 'CD19+',
       'CD25+', 'CD27+', 'CD27-', 'CD4+', 'CD45RA+', 'CD45RA-', 'CD45RO+',
       'CD45RO-', 'CD5+', 'CD5-', 'CD8+', 'CRTH2+', 'CXCR3+', 'CXCR3-',
       'SubSet', 'cell_type', 'log1p_tpm_rescaled_type',
       'log1p_tpm_rescaled_subset', 'log1p_tpm_rescaled', 'gene_cat',
       'gene_id', 'B_cell', 'T_cell'],
      dtype='object')


## sample genes for analysis

In [7]:
sample_df = cache.cached(models.prep_sample_df, sample_n=sample_n)

INFO:stancache.stancache:prep_sample_df: cache_filename set to prep_sample_df.cached.sample_n_1000.pkl


INFO:stancache.stancache:prep_sample_df: Loading result from cache


## fit model

In [8]:
stan_data = models.prep_stan_data(sample_df, by=by)

In [9]:
model_file = models.get_model_file(model_name=model_name)
print(cache._read_file(model_file))

## neg binom parameterization
## estimate correlation matrix among cell types
data {
    // dimensions
    int<lower=1> N;  // N obs
    int<lower=1> G;  // N genes
    int<lower=1> S;  // N samples
    int<lower=0> C;  // N classes (e.g. B-cell, T-cell, B_Naive, CD5, CD45RO, etc)
                     //     note: classes should be mutually exclusive. Each row here should sum to 1
    // int<lower=0> M; // number of cell-level predictors 
   
    // data for each gene*sample
    int<lower=1, upper=G> gene[N];    // gene id for each obs
    int<lower=1, upper=S> sample[N];  // sample id for each obs
    vector<lower=0, upper=1>[C] x[N]; // map each obs to each class (0:'- or ?', 1:'+')
    int<lower=0> y[N];                // count/tpm for each obs
    
    // group-level predictors for each class C
    // (to come) - 
}
transformed data {
    int sample_y[S, G];    // array (size SxG) of ints
    vector[C] sample_x[S]; // array (size S) of vectors[C]
    for (n in 1:N) {
        sample

In [10]:
model_fit = models.cached_stan_fit(file=model_file, data=stan_data, model_name=model_name)

INFO:stancache.stancache:Step 1: Get compiled model code, possibly from cache
INFO:stancache.stancache:StanModel: cache_filename set to model5.cython_0_25_1.model_code_5344338527.pystan_2_12_0_0.stanmodel.pkl
INFO:stancache.stancache:StanModel: Loading result from cache
INFO:stancache.stancache:Step 2: Get posterior draws from model, possibly from cache
INFO:stancache.stancache:sampling: cache_filename set to model5.cython_0_25_1.model_code_5344338527.pystan_2_12_0_0.stanfit.chains_4.data_88845472780.iter_2000.seed_1245502385.pkl
INFO:stancache.stancache:sampling: Starting execution
Process ForkPoolWorker-14:
Process ForkPoolWorker-19:
Process ForkPoolWorker-8:
Process ForkPoolWorker-10:
Process ForkPoolWorker-18:
Process ForkPoolWorker-20:
Process ForkPoolWorker-5:
Process ForkPoolWorker-15:
Process ForkPoolWorker-16:
Process ForkPoolWorker-17:
Process ForkPoolWorker-12:
Process ForkPoolWorker-7:
Process ForkPoolWorker-6:
Process ForkPoolWorker-13:
Process ForkPoolWorker-9:
Process Fo

KeyboardInterrupt: 

## check convergence (superficially)

In [11]:
models.print_stan_summary(model_fit, pars='lp__')

NameError: name 'model_fit' is not defined

In [ ]:
models.plot_stan_summary(model_fit, pars='theta', metric='Rhat')

## summarize posterior draws of theta by gene

In [ ]:
# meta-data used for plotting functions below
# so that the following code is invariant to the model run
colnames = list(stan_data['x'].columns)
sort_by = colnames[0]
print(sort_by)

In [ ]:
theta_ldf = models.prep_theta_summary(model_fit, sample_df=sample_df, colnames=colnames, expose_group=sort_by)

In [ ]:
## show theta estimates for first 50 genes, by `sort-by`
g = sns.boxplot(data=theta_ldf.loc[theta_ldf['mean_value_rank_{}'.format(sort_by)] <= 50,:] \
                .sort_values('mean_value_rank_{}'.format(sort_by)),
            y='new_gene_cat',
            x='value',
            hue='variable', 
            fliersize=0, width=2, linewidth=0.2)

In [ ]:
## zoom in on the highest-ranked genes by `sort-by` difference from average 
## across all cell types
g = sns.boxplot(data=theta_ldf.loc[theta_ldf['mean_abs_diff_rank_{}'.format(sort_by)] <= 10,:] \
                .sort_values('mean_diff_rank_{}'.format(sort_by)),
            y='new_gene_cat',
            x='value',
            hue='variable', 
            fliersize=0, linewidth=0.2)

## posterior-predictive checking for selected genes

In [ ]:
# identify top_genes by name
top_genes = theta_ldf.loc[theta_ldf['mean_abs_diff_rank_{}'.format(sort_by)] <= 10,:] \
                .drop_duplicates(subset='new_gene_cat')['new_gene_cat'].values
print(top_genes)

In [ ]:
# get yrep draws
yrep_df = models.prep_yrep_summary(model_fit, sample_df=sample_df, filter_genes=top_genes)

In [ ]:
models.plot_posterior_predictive_checks(model_fit=model_fit, plot_genes=top_genes, sample_df=sample_df,
                                        yrep_df=yrep_df, n_genes=1)

In [ ]:
models.plot_posterior_predictive_checks(model_fit=model_fit, plot_genes=top_genes[1:], sample_df=sample_df,
                                        yrep_df=yrep_df, n_genes=1)

In [ ]:
models.plot_posterior_predictive_checks(model_fit=model_fit, plot_genes=top_genes[2:], sample_df=sample_df,
                                        yrep_df=yrep_df, n_genes=1)

## summarize posterior draws for `theta_mu`

In [ ]:
mu_ldf = models.prep_theta_mu_summary(stan_fit=model_fit, stan_data=stan_data, par='theta_mu')

In [ ]:
a = sns.boxplot(data=mu_ldf, x='variable', y='value')
plt.setp(a.get_xticklabels(), rotation='vertical')

## summarize posterior draws for `Omega`

In [ ]:
omega_summary = models.prep_omega_summary(stan_fit=model_fit, stan_data=stan_data, par='Omega', gene_id=by)

In [ ]:
with sns.plotting_context('paper'):
    sns.heatmap(omega_summary.loc[:, list(stan_data['x'].columns)])